##SCRAPING
  Scarichiamo tweets per analizzare il sentiment di ognuno e metterlo in relazione all'andamento dell cryptovalute

In [ ]:
!pip install --user tweepy==3.3.0

In [ ]:
!pip install credentials

In [ ]:
import tweepy
import numpy as np
import pandas
from tweepy import OAuthHandler
from tqdm import tqdm_notebook as tqdm 


In [ ]:
consumer_key = 'Vzk6O5t967f1n4iz9iQcsQwhf'
consumer_secret = 'Q8sncdY1j85mwNitOqb5Spcg6U6uuZ0E2KNHHZEYF2J4DdCyVi'
access_token = '164224789-mFmU5YhessCNvubNojgfL605tbU0vh0NOna3HdQU'
access_secret = 'Ic5LNOH2QqxziSq4eTQdJvbXo3noTcQDUzCKXgFcdPzpC'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
userID = "?"

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

api = tweepy.API(auth)

tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

#Estrapolazione dei tweet e divisione degli ID


In [ ]:
for info in tweets[:50]:      
     print("ID: {}".format(info.id))
     print(info.created_at)
     print(info.full_text)
     print("\n")

#Raccolta tweet


In [ ]:
all_tweets = []             
all_tweets.extend(tweets)
oldest_id = tweets[-1].id
while True:
    tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           max_id = oldest_id - 1,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

In [ ]:
from pandas import DataFrame
outtweets = [[#tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx,tweet in enumerate(all_tweets)]
df = DataFrame(outtweets,columns=["created_at","favorite_count","retweet_count", "text"])
df.to_csv('%s_tweets.csv' % userID,index=False)
df.head(3)

In [ ]:
df.info()

#Creazione grafico 


In [ ]:
import matplotlib.pyplot as plt

ylabels = ["favorite_count","retweet_count"]

fig = plt.figure(figsize=(13,3))
fig.subplots_adjust(hspace=0.01,wspace=0.01)

n_row = len(ylabels)
n_col = 1
for count, ylabel in enumerate(ylabels):
    ax = fig.add_subplot(n_row,n_col,count+1)
    ax.plot(df["created_at"],df[ylabel])
    ax.set_ylabel(ylabel)
plt.show()